In [7]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import os, random, glob, ntpath
import create_mixed_audio_file_with_soundfile
# from create_mixed_audio_file_with_soundfile import combine_audio

speech_folder = {
    "train": "data/train/speech",
    "test": "data/test/speech",
}
noise_folder = {
    "train": "data/train/noise",
    "test": "data/test/noise"
}
mixed_folder = {
    "train": "data/train/mixed",
    "test": "data/test/mixed"
}

def run_combine(mode: str, snr: int, rnd_seed = None):
    if rnd_seed is not None:
        random.seed(rnd_seed)
    speech_list = glob.glob(speech_folder["train"] + "/*.wav")
    noise_list = glob.glob(noise_folder["train"] + "/*.wav")

    for speech_file in speech_list:
        noise_file = random.choice(noise_list)

        speech_name = ntpath.basename(speech_file).removesuffix(".wav")
        noise_name = ntpath.basename(noise_file).removesuffix(".wav")
        output_path = f"{mixed_folder["train"]}/{speech_name}_{noise_name}.wav"

        create_mixed_audio_file_with_soundfile.combine_audio(speech_file, noise_file,snr,output_path)

In [13]:

run_combine("train",-10)